### Training the model

In [1]:
import IPython
from classes.vqa_model_trainer import VqaModelTrainer
from common.model_utils import get_trainable_params_distribution
from common.functions import get_highlighted_function_code
from common.settings import data_access
from common.utils import VerboseTimer
from data_access.model_folder import ModelFolder

Using TensorFlow backend.


In [2]:
import logging
import vqa_logger 
logger = logging.getLogger(__name__)

In [4]:
model_location = 'C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190222_1346_47\\'
model_folder = ModelFolder(model_location)
model_folder

ModelFolder(folder=C:\Users\Public\Documents\Data\2019\models\20190222_1346_47)

### Loading the model to train:

In [5]:
epochs = 1
batch_size = 75
mt = VqaModelTrainer(model_folder, use_augmentation=True,batch_size=batch_size, data_access=data_access)

[2019-02-22 13:47:51][DEBUG] Loading Model: 0:00:03.007405
[2019-02-22 13:47:51][DEBUG] Loading Meta: 0:00:00.175097
[2019-02-22 13:47:51][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 13:47:51][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 13:47:54][DEBUG] Loading parquet: 0:00:02.850558
[2019-02-22 13:47:54][DEBUG] Converting to pandas: 0:00:00.024234
[2019-02-22 13:47:54][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 13:47:54][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 13:47:55][DEBUG] Loading parquet: 0:00:00.518445
[2019-02-22 13:47:55][DEBUG] Converting to pandas: 0:00:00.004960


#### Lets take a look at the parameters:

In [5]:
get_trainable_params_distribution(mt.model)
# mt.model.summary()

Got a total of 127,207 trainable parameters


,index,layer,trainable_params,pretty_value
0,7,dense_1/kernel:0,81920,"81,920"
1,6,model_output_tanh_dense/kernel:0,33904,"33,904"
2,3,embedding_batch_normalization/gamma:0,4608,"4,608"
3,5,embedding_batch_normalization/beta:0,4608,"4,608"
4,0,model_output_tanh_dense/bias:0,2119,"2,119"


#### And a look at data:

In [6]:
mt.df_meta_answers
mt.df_meta_words
mt.df_meta_answers.tail(2)

,answer
1706,pulmonary sequestration
1707,vestibular (acoustic) schwannoma


In [8]:
logger.debug(f"train Shape: {mt.data_train.shape}")
logger.debug(f"validation Shape: {mt.data_val.shape}")
mt.data_train.head(0)

[2019-02-22 12:49:51][DEBUG] train Shape: (12792, 12)
[2019-02-22 12:49:51][DEBUG] validation Shape: (2000, 12)


,index,image_name,question,answer,path,processed_question,processed_answer,diagnosis,question_category,answer_embedding,question_embedding,group


### Overview of preperations for training:

##### The functions for getting the features & labels:

In [9]:
from common.functions import get_features, sentences_to_hot_vector, hot_vector_to_words
code_get_labels = get_highlighted_function_code(mt.get_labels, remove_comments=True)


code_get_features = get_highlighted_function_code(get_features, remove_comments=True)
code_hot_vector = get_highlighted_function_code(sentences_to_hot_vector, remove_comments=True)


print('Getting the label using a hot vector\n')
IPython.display.display(code_get_labels)
print('\n\nThe underlying method:\n')
IPython.display.display(code_hot_vector)


print('\n\nGetting the features using question embeding concatenation:\n')
IPython.display.display(code_get_features)

Getting the label using a hot vector





The underlying method:





Getting the features using question embeding concatenation:



##### Example of hot vector of answer (AKA answer...)

In [10]:
df = mt.data_train

class_df = mt.class_df
class_count = len(class_df)
# class_df.sample(5)

classes_indices_df = [class_df.loc[class_df.isin(ans.lower().split())] for ans in  df.answer]
classes_indices = [list(d.index) for d in classes_indices_df]

idx_sample = 9
print(df.answer[idx_sample])
classes_indices[idx_sample]

t2


[]

##### Will transform the sentences into vector and back using the following:

In [11]:
code = get_highlighted_function_code(hot_vector_to_words,remove_comments=False)
IPython.display.display(code)  

##### Check it looks sane by inversing the binarizing:

In [12]:
# words = mt.df_meta_words.word

arr_one_hot_vector = mt.get_labels(mt.data_train)
categorial_labels = arr_one_hot_vector

idx = 0
answer =  mt.data_train.answer.loc[idx]
print(f'The sentence:\n{answer}')

one_hot_vector = arr_one_hot_vector[idx]
label_words = hot_vector_to_words(one_hot_vector, mt.class_df)
print('\n\nThe reversed answer from hot vector:')
label_words

The sentence:
cta - ct angiography


The reversed answer from hot vector:


0                ct
20              cta
1537    angiography
Name: word, dtype: object

In [13]:
history = mt.train()

[2019-02-22 12:50:43][DEBUG] Getting 2000 validation features: 0:00:16.860666
[2019-02-22 12:50:44][DEBUG] Getting 2000 validation labels: 0:00:00.209300
[2019-02-22 12:50:44][DEBUG] loading processed data from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 12:50:44][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\model_input.parquet
[2019-02-22 12:50:46][DEBUG] Loading parquet: 0:00:02.802816
[2019-02-22 12:50:46][DEBUG] Converting to pandas: 0:00:00.016047
[2019-02-22 12:50:46][DEBUG] Loading augmentations:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\augmentations.parquet
[2019-02-22 12:50:46][DEBUG] loading parquet from:
C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\data\augmentations.parquet
[2019-02-22 12:50:47][DEBUG] Loading parquet: 0:00:00.021333
[2019-02-22 12:50:47][DEBUG] Converting to pandas: 0:00:00.008826


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long long'


[2019-02-22 12:50:48][DEBUG] ===========================================================================
[2019-02-22 12:50:48][DEBUG] Expected shape: [(None, 4608, 1), (None, None, None, 3)]
[2019-02-22 12:50:48][DEBUG] ---------------------------------------------------------------------------
[2019-02-22 12:50:48][DEBUG] Actual training shape:((75, 4608, 1), (75, 224, 224, 3))
[2019-02-22 12:50:48][DEBUG] Actual Validation shape:((2000, 4608, 1), (2000, 224, 224, 3))
[2019-02-22 12:50:48][DEBUG] ---------------------------------------------------------------------------
[2019-02-22 12:50:48][DEBUG] Train Labels shape:(75, 2119)
[2019-02-22 12:50:48][DEBUG] Validation Labels shape:(2000, 2119)
[2019-02-22 12:50:48][DEBUG] ===========================================================================
Epoch 1/1
1705/1705 [==============================] - 1642s 963ms/step - loss: -0.4215 - f1_score: nan - recall_score: nan - precision_score: 0.1536 - acc: 0.2998 - val_loss: -0.4545 - val_f

C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\VQA.Python\common\model_utils.py:58: RuntimeWarning: Early stopping requires accuracy available!
  warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)


[2019-02-22 13:18:13][DEBUG] Training Model: 0:27:29.132835


### Save trained model:

In [14]:
with VerboseTimer("Saving trained Model"):
    model_folder = mt.save(mt.model, mt.model_folder, history)


[2019-02-22 13:19:24][DEBUG] model saved
[2019-02-22 13:19:24][DEBUG] saving prediction vector
[2019-02-22 13:19:24][DEBUG] saved prediction vector
[2019-02-22 13:19:24][DEBUG] Writing Summary
[2019-02-22 13:19:24][DEBUG] Done Writing Summary
[2019-02-22 13:19:24][DEBUG] Saving image
[2019-02-22 13:19:28][DEBUG] Image saved ('C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\model.png')
[2019-02-22 13:19:28][DEBUG] Saving History
[2019-02-22 13:19:28][DEBUG] History saved to 'C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\model_history.pkl'
[2019-02-22 13:19:28][DEBUG] Saving trained Model: 0:00:08.050156
[2019-02-22 13:19:28][INFO] Summary: C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\model_summary.txt
Image: C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\model.png
History: C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\model_history.pkl

[2019-02-22 13:19:28][INFO] model_location = 'C:\Users\Public\Documents\Data\2019\mo

In [15]:
print (model_folder.model_path)

C:\Users\Public\Documents\Data\2019\models\20190222_1319_20\vqa_model.h5
